In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet")

In [3]:
df.sample(10)

,price_subtotal,company_id,id,quantity,product_id_num,product_name,invoice_id_num,invoice_name,date_invoice
59673,47400.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1540960,1.0,19013,[BCS00025125] GS025 FILTRO COMBUSTIBLE CUMMINS...,418423,PVE52866 Mostrador Medellín T1/29243,2025-05-28 15:51:54
44299,30600.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1558456,1.0,40625,[DCS00923139] GS923 FILTRO SEPARADOR COMBUSTIB...,423297,PV4E15386 Mostrador Norte T1/17460,2025-06-26 17:00:52
142908,133200.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1441892,1.0,18908,[BLS00286125] GS286 FILTRO ACEITE DUAL MOTORES...,392450,PV3E16437 Mostrador Valladolid T2/13241,2024-12-27 17:11:19
178939,698400.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1398455,12.0,33103,[DAP02020025] DA2020 FILTRO AIRE DONSSON - RET...,381181,FCOT21335,2024-10-21 16:04:19
33639,41500.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1572489,1.0,17372,"[DAE03266025] DA3266 FILTRO AIRE KUBOTA,YAMAHA...",426684,FCOT25317,2025-07-16 13:39:36
114783,99000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1475145,1.0,18682,[BLS00202125] GS202 FILTRO ACEITE FILTROFILTRO...,401175,FCOT23122,2025-02-19 16:30:35
147500,60000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1435474,4.0,33107,[DCE00905189] G905 FILTRO COMBUSTUBLE ELEMNTO ...,391014,FCOT22252,2024-12-17 16:52:58
21964,110300.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1586010,1.0,16880,"[DAR02191025] DA2191 FILTRO AIRE 1o. A. COPCO,...",430480,PVE53642 Mostrador Medellín T1/29656,2025-08-05 15:35:10
74608,88100.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1523127,1.0,20813,[BCE00782125] G782 FILTRO COMBUSTIBLE HITACHI ...,413817,PVE52557 Mostrador Medellín T2/5174,2025-05-02 20:37:10
118561,523200.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1470690,12.0,18844,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",399920,FCAL3631,2025-02-13 14:53:22


In [21]:
import re
import unicodedata

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "CALI",
    "FMDE": "MEDELLIN",
    "FBOG": "BOGOTA",
    "FCTG": "CARTAGENA",
    "FBAQ": "BARRANQUILLA",
    "FVAL":"VALLADOLID",
    "FCOT":"COTA",
    "FBUC":"BUCARAMANGA",
    "FNOR":"NORTE"
}


# Equivalencias para normalizar nombres truncados o mal escritos
mapa_equivalencias = {
    "MEDELLIN": "MEDELLIN",
    "MEDELLI": "MEDELLIN",
    "MEDELL": "MEDELLIN",
    "MEDELI": "MEDELLIN",
    "CALI": "CALI",
    "BARRANQUILLA": "BARRANQUILLA",
    "BOGOTA": "BOGOTA",
    "CARTAGENA": "CARTAGENA",
    "VALLADOLID": "VALLADOLID"
}

def normalizar(texto):
    """Quita tildes y pasa a mayúsculas"""
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto.upper()

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    sucursal = None
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip()
        else:
            # 3) Buscar prefijo de código
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # Normalizar texto
    sucursal = normalizar(sucursal)
    
    # Limpiar T1, T2, T3 al final
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    # Aplicar equivalencias
    sucursal = mapa_equivalencias.get(sucursal, sucursal)
    
    return sucursal

# Aplicar al dataframe
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

In [50]:
df.sample(5)

,price_subtotal,company_id,id,quantity,product_id_num,product_name,invoice_id_num,invoice_name,date_invoice,Sucursal
95887,32300.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1496808,1.0,17684,[DAB04783025] DA4783 FILTRO AIRE SEGURIDAD- C...,407083,PV4E14532 Mostrador Norte T2/11934,2025-03-25 20:10:17,NORTE
116648,36000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1472999,1.0,17330,[DAR03066025] DA3066 FILTRO AIRE 1o. A.CHALLME...,400640,PV1E9375 Mostrador Cota T2/2877,2025-02-17 16:28:51,COTA
74210,24600.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1523554,1.0,29464,[DCE00843189] G843 FILTRO COMBUSTIBLE CHEVROLE...,413958,FNOR1580,2025-05-05 12:30:17,NORTE
143802,234800.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1440802,4.0,19241,[BCS00427125] GS427 FILTRO COMB. SEPARAD. SECU...,392106,PV4E13737 Mostrador Norte T2/11507,2024-12-23 18:46:54,NORTE
83030,85700.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1513613,1.0,19155,[BCS00087125] GS087 FILTRO COMBUSTIBLE CUMMINS...,411155,FBAQ13107,2025-04-15 18:20:14,BARRANQUILLA


In [23]:
ventas_sucursal = df.groupby("Sucursal")["quantity"].sum()

In [24]:
ventas_sucursal.head(10)

Sucursal
BARRANQUILLA         99503.0
BUCARAMANGA          64832.0
CALI                 46547.0
CALLE 6              58255.0
COTA                260199.0
MEDELLIN             93158.0
NORTE                23292.0
VALLADOLID           39799.0
VENDEDOR EXTERNO      6711.0
Name: quantity, dtype: float64